<a href="https://colab.research.google.com/github/sheikmohdimran/Experiments_2020/blob/master/NLP/TTS_AudioBook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
!git clone https://github.com/TensorSpeech/TensorFlowTTS
os.chdir("TensorFlowTTS")
!pip install  .
os.chdir("..")
import sys
sys.path.append("TensorFlowTTS/")

Cloning into 'TensorFlowTTS'...
remote: Enumerating objects: 4945, done.
remote: Total 4945 (delta 0), reused 0 (delta 0), pack-reused 4945
Receiving objects: 100% (4945/4945), 81.43 MiB | 20.40 MiB/s, done.
Resolving deltas: 100% (3152/3152), done.
Processing /content/TensorFlowTTS
     |████████████████████████████████| 320.4MB 47kB/s 
     |████████████████████████████████| 1.1MB 46.6MB/s 
     |████████████████████████████████| 184kB 53.0MB/s 
     |████████████████████████████████| 245kB 48.7MB/s 
     |████████████████████████████████| 225kB 52.0MB/s 
     |████████████████████████████████| 788kB 49.4MB/s 
     |████████████████████████████████| 1.7MB 42.7MB/s 
     |████████████████████████████████| 3.1MB 47.7MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 184kB 58.9MB/s 
  Created wheel for TensorFlowTTS: filename=TensorFlowTTS-0.0-cp36-none-any.whl size=112618 sha256=0405e94ad61ed903108a3d3a6d4baeb606378a82c9d3e18e902edc86b85

In [2]:
print("Downloading Tacotron2 model...")
!gdown --id {"12jvEO1VqFo1ocrgY9GUHF_kVcLn3QaGW"} -O tacotron2-120k.h5
!gdown --id {"1OI86hkN1YCpHBsIKnkELNbSho5Pj-pPY"} -O tacotron2_config.yml

Downloading...
From: https://drive.google.com/uc?id=12jvEO1VqFo1ocrgY9GUHF_kVcLn3QaGW
To: /content/tacotron2-120k.h5
128MB [00:01, 93.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1OI86hkN1YCpHBsIKnkELNbSho5Pj-pPY
To: /content/tacotron2_config.yml
100% 1.49k/1.49k [00:00<00:00, 2.41MB/s]


In [3]:
print("Downloading Multi-band MelGAN model...")
!gdown --id {"1kChFaLI7slrTtuk3pvcOiJwJDCygsw9C"} -O mb.melgan-940k.h5
!gdown --id {"1YC_kZpuRZGQ-JfMKj1LC0YRyKXsgLTJL"} -O mb.melgan_config.yml

Downloading...
From: https://drive.google.com/uc?id=1kChFaLI7slrTtuk3pvcOiJwJDCygsw9C
To: /content/mb.melgan-940k.h5
10.2MB [00:00, 47.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1YC_kZpuRZGQ-JfMKj1LC0YRyKXsgLTJL
To: /content/mb.melgan_config.yml
100% 2.18k/2.18k [00:00<00:00, 3.80MB/s]


In [4]:
import tensorflow as tf

import yaml
import numpy as np
import matplotlib.pyplot as plt

import IPython.display as ipd

from tensorflow_tts.inference import TFAutoModel
from tensorflow_tts.inference import AutoConfig
from tensorflow_tts.inference import AutoProcessor

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


In [5]:
tacotron2_config = AutoConfig.from_pretrained('TensorFlowTTS/examples/tacotron2/conf/tacotron2.v1.yaml')
tacotron2 = TFAutoModel.from_pretrained(
    config=tacotron2_config,
    pretrained_path="tacotron2-120k.h5",
    training=False, 
    name="tacotron2"
)

In [6]:
mb_melgan_config = AutoConfig.from_pretrained('TensorFlowTTS/examples/multiband_melgan/conf/multiband_melgan.v1.yaml')
mb_melgan = TFAutoModel.from_pretrained(
    config=mb_melgan_config,
    pretrained_path="mb.melgan-940k.h5",
    name="mb_melgan"
)

In [7]:
print("Downloading ljspeech_mapper.json ...")
!gdown --id {"1YBaDdMlhTXxsKrH7mZwDu-2aODq5fr5e"} -O ljspeech_mapper.json

Downloading...
From: https://drive.google.com/uc?id=1YBaDdMlhTXxsKrH7mZwDu-2aODq5fr5e
To: /content/ljspeech_mapper.json
100% 3.57k/3.57k [00:00<00:00, 2.89MB/s]


In [8]:
processor = AutoProcessor.from_pretrained(pretrained_path="./ljspeech_mapper.json")

In [9]:
def do_synthesis(input_text, text2mel_model, vocoder_model, text2mel_name, vocoder_name):
  input_ids = processor.text_to_sequence(input_text)

  # text2mel part
  if text2mel_name == "TACOTRON":
    _, mel_outputs, stop_token_prediction, alignment_history = text2mel_model.inference(
        tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
        tf.convert_to_tensor([len(input_ids)], tf.int32),
        tf.convert_to_tensor([0], dtype=tf.int32)
    )
  elif text2mel_name == "FASTSPEECH":
    mel_before, mel_outputs, duration_outputs = text2mel_model.inference(
        input_ids=tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
        speaker_ids=tf.convert_to_tensor([0], dtype=tf.int32),
        speed_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
    )
  elif text2mel_name == "FASTSPEECH2":
    mel_before, mel_outputs, duration_outputs, _, _ = text2mel_model.inference(
        tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
        speaker_ids=tf.convert_to_tensor([0], dtype=tf.int32),
        speed_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
        f0_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
        energy_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
    )
  else:
    raise ValueError("Only TACOTRON, FASTSPEECH, FASTSPEECH2 are supported on text2mel_name")

  # vocoder part
  if vocoder_name == "MELGAN" or vocoder_name == "MELGAN-STFT":
    audio = vocoder_model(mel_outputs)[0, :, 0]
  elif vocoder_name == "MB-MELGAN":
    audio = vocoder_model(mel_outputs)[0, :, 0]
  else:
    raise ValueError("Only MELGAN, MELGAN-STFT and MB_MELGAN are supported on vocoder_name")

  if text2mel_name == "TACOTRON":
    return mel_outputs.numpy(), alignment_history.numpy(), audio.numpy()
  else:
    return mel_outputs.numpy(), audio.numpy()


In [11]:
import re
with open('/content/1.txt', 'r') as reader:
    chapter = reader.read()

chapter=re.sub('\n\n', '\n', chapter)
chapter=re.sub('\t', '', chapter)
chapter=re.sub('—', '', chapter)
chapter_list=list(chapter.split("\n")) 
chapter_list=list(filter(None, chapter_list))

In [12]:
final=[]
for i in chapter_list:
  print((i))
  _, _, audios = do_synthesis(i, tacotron2, mb_melgan, "TACOTRON", "MB-MELGAN")
  final.append(audios)

Colin Hazelton made a show of checking the time on his mobile phone as the woman sat down. She was an hour late and he wanted to indicate his displeasure, even if only passive-aggressively.
She fixed the hem of her skirt and crossed her legs, and only then did she look up at him. She seemed to notice the phone and his focus on it, then she lifted the sweating water glass in front of her and took a sip.
Hazelton dropped his phone back into his pocket and drank down half of his gin and tonic. He had to admit she was every bit as attractive as advertised. It was virtually all his control had said about his contact tonight. Statuesque and blond, with mannerisms that transmitted refinement and poise. Still, Hazelton was too pissed to be impressed. Not pissed at her, exclusively, but generally angry, and he certainly wasn’t in the mood to ogle his contact tonight.
That she’d made him wait a goddamned hour took even more of the luster off her splendor.
Before either spoke the waiter appeared.

In [13]:
len(final)

116

In [14]:
ipd.Audio(final[1], rate=22050)

In [43]:
import soundfile as sf

fnames=[]
for i,j in enumerate(final):
  sf.write("sample%s.flac" %i, j, 22050)
  fnames.append("file '/content/sample%s.flac'" %i)

In [44]:
with open("outfile", "w") as outfile:
    outfile.write("\n".join(fnames))

In [46]:
!ffmpeg -f concat -safe 0 -i outfile final.flac

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [49]:
!rm -rf sample*.flac